## Traditional machine learning
Feature(s): 'SMILES' column
    translate to morgan fingerprint

target: mp_bin
    binary class of melting points

In [1]:
import sys
import os

# Append the parent directory of your package to sys.path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..', '..', '..', '..')))


Read the dataset from the zip file

In [2]:
import pandas as pd
import numpy as np
import zipfile

path_to_dataset = 'train_meltingPoint_noDuplicates.zip'
csv_filename = 'train_meltingPoint_noDuplicates.csv'

# Open the file, Correct the encoding and sep if necessary
if path_to_dataset.endswith('.zip'):
    with zipfile.ZipFile(path_to_dataset, 'r') as z:
        # Open the CSV file within the ZIP file
        with z.open(csv_filename) as f:
            # Read the CSV file into a DataFrame
            df = pd.read_csv(f, sep=',', on_bad_lines='warn')
else:
    # Read the CSV file into a DataFrame
    df = pd.read_csv(path_to_dataset, sep=',', on_bad_lines='warn')


print('Count of unique smiles:', df.SMILES.unique().shape[0])
print('Count of all of the smiles:', df.shape[0])



Count of unique smiles: 273237
Count of all of the smiles: 273237


In [3]:
from MLPipeline import MLmodel, BinTheTarget

Target = 'mp_bin'
Features = ['SMILES']
Feature_types = ['SMILES']
input = df[:7000]

/home/lsmo/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


The Random forest classifier model without hyperparameter optimization

In [4]:
model = MLmodel(modelType='RandomForestClassifier',
                    df=input,
                    target=Target,
                    features=Features,
                    feature_types=Feature_types)

# get the values (input and output) of the model
X_train, X_test, y_train, y_test = model.getValues()

[18:12:30] Explicit valence for atom # 10 S, 9, is greater than permitted
[18:12:32] Explicit valence for atom # 16 S, 9, is greater than permitted
[18:12:32] Explicit valence for atom # 10 S, 9, is greater than permitted


Some information about the dataset such as the splitting, wrong smiles, total number of samples.

In [6]:
number_of_samples, number_of_wrong_smiles, clean_df = model.getdfAnalysis(orginal_df=input)
print('Number of samples:', number_of_samples)
print('Number of wrong smiles:', number_of_wrong_smiles)


for i in range(len(clean_df.mp_bin.value_counts())):
    print(f'bin {i} fraction in original set:' , 
          clean_df.mp_bin.value_counts()[i]/clean_df.mp_bin.value_counts().sum())

for i in range (len(pd.DataFrame(y_train).value_counts())):
    print(f'bin {i} fraction in training set:' , 
          pd.DataFrame(y_train).value_counts()[i]/pd.DataFrame(y_train).value_counts().sum())

for i in range(len(pd.DataFrame(y_test).value_counts())):
    print(f'bin {i} fraction in test set:' , 
          pd.DataFrame(y_test).value_counts()[i]/pd.DataFrame(y_test).value_counts().sum())

bin 0 fraction in original set: 0.5149349721309132
bin 1 fraction in original set: 0.48506502786908673
bin 0 fraction in training set: 0.52
bin 1 fraction in training set: 0.48
bin 0 fraction in test set: 0.52
bin 1 fraction in test set: 0.48


In [7]:
model.train()
model.evaluate()

RandomForestClassifier model trained successfully.
Accuracy for RandomForestClassifier: 0.68


0.68

Train the random forest classifier using hyperparameter optimization with optuna

In [8]:
from sklearn.model_selection import cross_val_score
from sklearn.base import clone

def objective(trial, model_instance):
    """
    Objective function for Optuna to minimize.
    """
    # Define hyperparameters to tune
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 50, 300),
        'max_depth': trial.suggest_categorical('max_depth', [None, 10, 20, 30, 40]),
        'min_samples_split': trial.suggest_int('min_samples_split', 2, 15),
        'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 6),
        'max_features': trial.suggest_categorical('max_features', ['sqrt', 'log2']),
        'bootstrap': trial.suggest_categorical('bootstrap', [True, False])
    }

    # Clone the model to ensure a fresh instance each trial
    model_clone = clone(model_instance.model)
    model_clone.set_params(**params)
    
    # Define the score metric
    scoring = 'accuracy'

    # Perform cross-validation
    scores = cross_val_score(model_clone, model_instance.X_train, model_instance.y_train, cv=model_instance.cv, scoring=scoring)

    # Return the average score across all folds
    return scores.mean()

In [9]:
model = MLmodel(modelType='RandomForestClassifier', df=input, target=Target, 
                features=['SMILES'], hyperparameter_tuning=True,
                feature_types=Feature_types,
                optimization_method='optuna', objective=lambda trial: objective(trial, model))

model.train()
predictions = model.predict()
model.evaluate()

[18:12:36] Explicit valence for atom # 10 S, 9, is greater than permitted
[18:12:38] Explicit valence for atom # 16 S, 9, is greater than permitted
[18:12:39] Explicit valence for atom # 10 S, 9, is greater than permitted
[I 2024-09-11 18:12:40,274] A new study created in memory with name: no-name-e8d1e17b-ca87-4a31-8deb-c7b115a13cce
[I 2024-09-11 18:12:40,726] Trial 0 finished with value: 0.54 and parameters: {'n_estimators': 117, 'max_depth': 20, 'min_samples_split': 6, 'min_samples_leaf': 6, 'max_features': 'log2', 'bootstrap': False}. Best is trial 0 with value: 0.54.
[I 2024-09-11 18:12:41,723] Trial 1 finished with value: 0.58 and parameters: {'n_estimators': 185, 'max_depth': 20, 'min_samples_split': 8, 'min_samples_leaf': 3, 'max_features': 'log2', 'bootstrap': True}. Best is trial 1 with value: 0.58.
[I 2024-09-11 18:12:42,855] Trial 2 finished with value: 0.62 and parameters: {'n_estimators': 288, 'max_depth': 10, 'min_samples_split': 11, 'min_samples_leaf': 4, 'max_features'

RandomForestClassifier model trained successfully.
Accuracy for RandomForestClassifier: 0.64


0.64